In [9]:
import numpy as np # General numerics
from scipy.integrate import odeint # Integration
from scipy.optimize import minimize # Optimization
import matplotlib.pyplot as plt # Plotting

ns=16
x10 = 1 
x20 = 0 
x30 = 0 
w1=np.zeros(ns)
w2=np.zeros(ns)
w0=[]
for d in range(2*len(w1)):
    if d<len(w1):
        w0.append(w1[d])
    else:
        w0.append(w2[d-ns])
inte=2
for i in range(1,inte):
    t = np.linspace(0,1,ns+1)
    def state(y,t,w1,w2):
        x1,x2,x3 = y
        
        f = [ w2,
            -x1+w1,
            -6*x1-12*x2+3*w1+w2];
        
        return f
    def objective(w0):
        y0 = [x10,x20,x30]
        for ks in range(0, ns):
            sol = odeint(state,y0,np.linspace(t[ks],t[ks+1],20), args=(w0[ks],w0[ns+ks],), Dfun=None, col_deriv=0, full_output=0, ml=None, mu=None, rtol=None, atol=None, tcrit=None, h0=0.0, hmax=0.0, hmin=0.0, ixpr=0, mxstep=0, mxhnil=0, mxordn=12, mxords=5, printmessg=0, tfirst=False)
            y0=sol[-1]
        obj = sol[-1][2]
        return obj

    
    bnd=[]
    for o in range(0,2*ns): 
        if o < ns:
            bnd.append((-10,10))
        else:
            bnd.append((-10,10))
    res = minimize(objective,w0,method='SLSQP', jac=None, bounds=bnd, constraints=(), tol=None, callback=None,options={'maxiter': 10000,'disp':True})
    w0=[]
    for o in range(0,ns): 
        w0.append(res.x[o])
        w0.append(res.x[o])
    ns=ns*2

Optimization terminated successfully.    (Exit mode 0)
            Current function value: -41.30859394726245
            Iterations: 55
            Function evaluations: 1870
            Gradient evaluations: 55


In [17]:
import numpy as np # General numerics
from scipy.integrate import odeint # Integration
from scipy.optimize import minimize # Optimization
import matplotlib.pyplot as plt # Plotting
import pywt
import statistics

ns=16
x10 = 1 
x20 = 0 
x30 = 0 
w1=np.zeros(ns)
w2=np.zeros(ns)
w0=[]

for d in range(2*len(w1)):
    if d<len(w1):
        w0.append(w1[d])
    else:
        w0.append(w2[d-ns])
i=0
fun=1
inte=10
tg=[0]*inte
data2=[0]*inte
err=1
tol=1e-6
while err>tol:
    i=i+1
    t = np.linspace(0,1,ns+1)
    def state(y,t,w1,w2):
        x1,x2,x3 = y
        
        f = [ w2,
            -x1+w1,
            -6*x1-12*x2+3*w1+w2];
        
        return f
    def objective(w0):
        y0 = [x10,x20,x30]
        for ks in range(0, ns):
            sol = odeint(state,y0,np.linspace(t[ks],t[ks+1],20), args=(w0[ks],w0[ns+ks],), Dfun=None, col_deriv=0, full_output=0, ml=None, mu=None, rtol=None, atol=None, tcrit=None, h0=0.0, hmax=0.0, hmin=0.0, ixpr=0, mxstep=0, mxhnil=0, mxordn=12, mxords=5, printmessg=0, tfirst=False)
            y0=sol[-1]
        obj = sol[-1][2]
        return obj

    
    bnd=[]
    for o in range(0,2*ns): 
        if o < ns:
            bnd.append((-10,10))
        else:
            bnd.append((-10,10))
    res = minimize(objective,w0,method='SLSQP', jac=None, bounds=bnd, constraints=(), tol=None, callback=None,options={'maxiter': 10000,'disp':True})
    err=abs((res.fun-fun)/res.fun)
    print(err)
    fun=res.fun
    
    db1 = pywt.Wavelet('db1')
    maxlevel=pywt.dwt_max_level(len(res.x)/2, db1)
    
    cD1 = pywt.wavedec(res.x[0:ns], db1,mode='symmetric',level=maxlevel)
    cD2 = pywt.wavedec(res.x[ns:2*ns], db1,mode='symmetric',level=maxlevel)
    
    Cdvr1=statistics.pstdev(cD1[-1])
    Thr1=Cdvr1*(2*np.log(len(cD1[-1])))**(1/2)*0.5
    Cdvr2=statistics.pstdev(cD2[-1])
    Thr2=Cdvr2*(2*np.log(len(cD2[-1])))**(1/2)*0.5
    
    indx1 = np.where(abs(cD1[-1])>abs(Thr1))
    indx2 = np.where(abs(cD2[-1])>abs(Thr2))
    z=0
    for x in range(len(indx2[0])):
        if indx2[0][x]==0:
            t=np.insert(t,ns+indx2[0][x]*2+z+2,(t[ns+indx2[0][x]*2+z+1]*0.6+t[ns+indx2[0][x]*2+z+2]*0.4))
            w0=np.insert(w0,ns+indx2[0][x]*2+z+2,w0[ns+indx2[0][x]*2+z+1])
            z=z+1
        elif indx2[0][x]==len(cD2[-1]):
            t=np.insert(t,ns+indx2[0][x]*2+z-1,(t[ns+indx2[0][x]*2+z-2]*0.6+t[ns+indx2[0][x]*2+z-1]*0.4))
            w0=np.insert(w0,ns+indx2[0][x]*2+z-1,w0[ns+indx2[0][x]*2+z-2])
            z=z+1
        
        else:
            t=np.insert(t,ns+indx2[0][x]*2+z-1,(t[ns+indx2[0][x]*2+z-2]*0.6+t[ns+indx2[0][x]*2+z-1]*0.4))
            w0=np.insert(w0,ns+indx2[0][x]*2+z-1,w0[ns+indx2[0][x]*2+z-2])
            z=z+1
        
            t=np.insert(t,ns+indx2[0][x]*2+z+1,(t[ns+indx2[0][x]*2+z]*0.4+t[ns+indx2[0][x]*2+z+1]*0.6))
            w0=np.insert(w0,ns+indx2[0][x]*2+z+1,w0[ns+indx2[0][x]*2+z])
            z=z+1
    for x in range(len(indx1[0])):
        if indx1[0][x]==0:
            t=np.insert(t,indx1[0][x]*2+z+2,(t[indx1[0][x]*2+z+1]*0.6+t[indx1[0][x]*2+z+2]*0.4))
            w0=np.insert(w0,indx1[0][x]*2+z+2,w0[indx1[0][x]*2+z+1])
            z=z+1
        elif indx1[0][x]==len(cD[-1]):
            t=np.insert(t,indx1[0][x]*2+z-1,(t[indx1[0][x]*2+z-2]*0.6+t[indx1[0][x]*2+z-1]*0.4))
            w0=np.insert(w0,indx1[0][x]*2+z-1,w0[indx1[0][x]*2+z-2])
            z=z+1
        
        else:
            t=np.insert(t,indx1[0][x]*2+z-1,(t[indx1[0][x]*2+z-2]*0.6+t[indx1[0][x]*2+z-1]*0.4))
            w0=np.insert(w0,indx1[0][x]*2+z-1,w0[indx1[0][x]*2+z-2])
            z=z+1
        
            t=np.insert(t,indx1[0][x]*2+z+1,(t[indx1[0][x]*2+z]*0.4+t[indx1[0][x]*2+z+1]*0.6))
            w0=np.insert(w0,indx1[0][x]*2+z+1,w0[indx1[0][x]*2+z])
            z=z+1
        
    
    # Usei isso aqui somente para verificar a integração com o controle otimo
    y0 = [x10,x20,x30]
    for ks in range(ns):
            sol = odeint(state,y0,np.linspace(t[ks],t[ks+1],20), args=(w0[ks],w0[ns+ks],))
            y0=sol[-1]
    print(ns)
    #Plot u(t) vs tempo
    plt.figure(i)
    plt.step(np.linspace(0,1,ns),res.x[0:ns],'bo',LineWidth=2)
    plt.axis([0, 1, -10, 10])
    plt.xlabel("Time (h)")
    plt.ylabel("u(t)")
    plt.show()
    
    #atualização do ns
    ns=len(t)-1

Optimization terminated successfully.    (Exit mode 0)
            Current function value: -41.30859394726245
            Iterations: 55
            Function evaluations: 1870
            Gradient evaluations: 55
1.0242080377094576


IndexError: index 17 is out of bounds for axis 0 with size 17

In [19]:
t

array([0.    , 0.0625, 0.125 , 0.1875, 0.25  , 0.3125, 0.375 , 0.4375,
       0.5   , 0.5625, 0.625 , 0.6875, 0.75  , 0.8125, 0.875 , 0.9375,
       1.    ])